In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

from kamera import kamera

In [ ]:
n = int(1e4)
bounds = 3 * np.array([50 * 1e-3, 50 * 1e-3, 50 * 1e-3, 20 * np.pi / 180, 20 * np.pi / 180, 20 * np.pi / 180])
output_data = bounds * (np.random.rand(n, 6) - 0.5) * 2
input_data = kamera(output_data)

In [ ]:
# rozdělit data na trénovací a testovací set
X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.1, random_state=42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        #self.fc5 = nn.Linear(16, 16)
        #self.fc6 = nn.Linear(16, 16)
        #self.fc7 = nn.Linear(16, 16)
        self.fc8 = nn.Linear(512, output_size)
        self.relu = nn.Tanh()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        #x = self.relu(self.fc5(x))
        #x = self.relu(self.fc6(x))
        #x = self.relu(self.fc7(x))
        x = self.fc8(x)
        return x


# Assuming X_train, X_test, y_train are already defined somewhere in your code
# Create the model and transfer it to the GPU
model = Net(X_train.shape[1], y_train.shape[1])

In [ ]:
import h5py
# Load the model weights from HDF5 file
with h5py.File('model_weights13.h5', 'r') as h5file:
    for name, param in model.named_parameters():
        # Ensure the parameter name matches the HDF5 dataset name structure
        param.data.copy_(torch.from_numpy(h5file[name][...]))


In [ ]:
model.to(device)

In [ ]:
std = np.array([0.0028, 0.0015, 0.0094, 0.0083, 0.0092, 
                0.0074, 0.0089, 0.0072, 0.0089, 0.0083])
std

In [ ]:
device = 'cpu'

In [ ]:
for trial_sets in range(100000):

    n = int(1e5)
    bounds = 1.05*np.array([50 * 1e-3, 50 * 1e-3, 50 * 1e-3, 20 * np.pi / 180, 20 * np.pi / 180, 20 * np.pi / 180])
    output_data = bounds * (np.random.rand(n, 6) - 0.5) * 2
    input_data = kamera(output_data) + np.random.normal(scale=std, size=(n, 10))*0
    # rozdělit data na trénovací a testovací set
    X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.1, random_state=42)

    # Convert data to PyTorch tensors and send to the device (GPU if available)
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

    # Define the loss function and the optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.000002)  # Decrease the learning rate

    # Add a learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1000, factor=0.5)

    # Assuming all previous setup code remains the same and tensors (X_train_tensor, y_train_tensor, X_test_tensor)
    # are already transferred to the appropriate device (GPU or CPU)

    # No need for DataLoader as we will not use mini-batches

    # Train the model without using batches
    num_epochs = 1000
    best_loss = np.inf
    patience, trials = 2000, 0

    for epoch in range(num_epochs):
        # Ensure the entire dataset is on the same device as the model

        # Forward pass
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Adjust the learning rate based on the loss
        scheduler.step(loss)

        if epoch % 100 == 0:
            outputs = model(X_test_tensor)
            loss2 = criterion(outputs, y_test_tensor)
            print('Epoch [{}/{}], Loss: {:.10f}, test Loss reached {:.10f}, min Loss reached {:.10f}, stagnating for {} it.'.format(epoch +
                1, num_epochs, loss.item(),loss2.item(), best_loss, trials))
            if loss2.item() > 10*best_loss:
                print(f"Overfitting detected at epoch {epoch+1}")
                break

        # Early stopping
        if loss.item() < best_loss:
            best_loss = loss.item()
            trials = 0
        else:
            trials += 1
            if trials >= patience:
                print(f'Stopping early at epoch {epoch+1}')
                break

In [ ]:
current_lr = optimizer.param_groups[0]['lr']
print(f"Current learning rate: {current_lr}")

In [ ]:
device = 'cpu'

In [ ]:
model.to(device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

In [ ]:
res_surr = y_pred_tensor.cpu().numpy()

In [ ]:
diff = y_test - res_surr

In [ ]:
plt.hist(np.abs(diff)[:,1], bins=100)
np.max(np.abs(diff), axis=0)

In [ ]:
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_train_tensor)
res_surr = y_pred_tensor.cpu().numpy()
diff = y_train - res_surr
np.max(np.abs(diff).ravel())

In [ ]:
import h5py

# If your model is on CUDA, move it back to CPU
model.to('cpu')

# Assuming the model is trained here and ready to be saved

# Save model parameters to HDF5, including biases
with h5py.File('model_weights15.h5', 'w') as h5file:
    for name, param in model.state_dict().items():
        h5file.create_dataset(name, data=param.numpy())

In [ ]:
model.to(device)

In [ ]:
# load matlab saved file noisy_data.mat and print what variables are in it
import scipy.io
mat = scipy.io.loadmat('noisy_data.mat')
print(mat.keys())
# clean the double underscores from the variable names
mat = {k.strip('_'): v for k, v in mat.items()}

# load rest of the variables in mat into dictionary
mat = {k: v for k, v in mat.items() if not k.startswith('__')}

# create torch dataloader on the noisy data its synth_x_best as the output and synth_y_test as the input
input_data_noisy = torch.tensor(mat['synth_y_test'], dtype=torch.float32)
output_data_noisy = torch.tensor(mat['synth_x_best'], dtype=torch.float32)
dataloader = DataLoader(TensorDataset(input_data_noisy, output_data_noisy), batch_size=100000, shuffle=True)

In [ ]:
for batch_input, batch_output in dataloader:

    X_train, X_test, y_train, y_test = train_test_split(batch_input, batch_output, test_size=0.1, random_state=42)

    # Convert data to PyTorch tensors and send to the device (GPU if available)
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

    # Define the loss function and the optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0000001)  # Decrease the learning rate

    # Add a learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1000, factor=0.5)

    # Assuming all previous setup code remains the same and tensors (X_train_tensor, y_train_tensor, X_test_tensor)
    # are already transferred to the appropriate device (GPU or CPU)

    # No need for DataLoader as we will not use mini-batches

    # Train the model without using batches
    num_epochs = 1000
    best_loss = np.inf
    patience, trials = 2000, 0

    for epoch in range(num_epochs):
        # Ensure the entire dataset is on the same device as the model

        # Forward pass
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Adjust the learning rate based on the loss
        scheduler.step(loss)

        if epoch % 100 == 0:
            outputs = model(X_test_tensor)
            loss2 = criterion(outputs, y_test_tensor)
            print('Epoch [{}/{}], Loss: {:.10f}, test Loss reached {:.10f}, min Loss reached {:.10f}, stagnating for {} it.'.format(epoch +
                1, num_epochs, loss.item(),loss2.item(), best_loss, trials))
            if loss2.item() > 10*best_loss:
                print(f"Overfitting detected at epoch {epoch+1}")
                break

        # Early stopping
        if loss.item() < best_loss:
            best_loss = loss.item()
            trials = 0
        else:
            trials += 1
            if trials >= patience:
                print(f'Stopping early at epoch {epoch+1}')
                break